# **Classification Problem**
## I'm trying to predict the case verdict by segregating into 3 categories-- convicted, acquitted and others based on case type(depending on the crime) , district and state(to show the prevalent beliefs and constructs of the region) , all genders(in case of any gender bias) and the position of the judge. For lesser running time per notebook, I separated neural network training and traditional machine learning models. This notebook is where I train a neural network which eventually achieved 0.9616 accuracy.

## **Importing relevant modules**

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

## **Reading relevant CSVs**
### Case files from 2010-2015

In [4]:
case=pd.read_csv("/kaggle/input/precog-cases/cases_2010.csv")
for i in range(1,5):
    case_year=pd.read_csv("/kaggle/input/precog-cases/cases_201%s.csv" %i)
    case=pd.concat([case, case_year])
    

### (Judges vs Case) and Judges Gender

In [5]:
judges= pd.read_csv("/kaggle/input/keys-precog/judge_case_merge_key.csv")
judge_gender= pd.read_csv("/kaggle/input/judges-clean/judges_clean.csv")

In [6]:
#renaming judge_gender column name for merging later
judge_gender = judge_gender.rename(columns={'ddl_judge_id': 'ddl_decision_judge_id'})
judge_gender

,ddl_decision_judge_id,state_code,dist_code,court_no,judge_position,female_judge,start_date,end_date
0,1,1,1,1,chief judicial magistrate,0 nonfemale,20-09-2013,20-02-2014
1,2,1,1,1,chief judicial magistrate,0 nonfemale,31-10-2013,20-02-2014
2,3,1,1,1,chief judicial magistrate,0 nonfemale,21-02-2014,31-05-2016
3,4,1,1,1,chief judicial magistrate,0 nonfemale,01-06-2016,06-06-2016
4,5,1,1,1,chief judicial magistrate,0 nonfemale,06-06-2016,07-07-2018
...,...,...,...,...,...,...,...,...
98473,98474,30,2,9,criminal cases,1 female,21-04-2004,14-11-2013
98474,98475,30,2,9,criminal cases,1 female,16-01-2015,16-01-2016
98475,98476,30,2,9,criminal cases,1 female,09-12-2016,31-07-2017
98476,98477,30,2,10,criminal cases,1 female,15-05-2017,28-01-2019


### Merging judges and cases to get judges corresponding to each case

In [7]:
judge_case=pd.merge(judges,case,on='ddl_case_id')
judge_case


,ddl_case_id,ddl_filing_judge_id,ddl_decision_judge_id,year,state_code,dist_code,court_no,cino,judge_position,female_defendant,...,female_adv_def,female_adv_pet,type_name,purpose_name,disp_name,date_of_filing,date_of_decision,date_first_list,date_last_list,date_next_list
0,01-01-01-201908000012013,50.0,50.0,2013,1,1,1,MHNB030000112013,chief judicial magistrate,0 male,...,-9999,0,1919.0,7062.0,25,2013-01-01,2013-01-01,2013-01-01,2013-01-01,2013-01-01
1,01-01-01-201908000022014,92.0,93.0,2014,1,1,1,MHNB030000982014,chief judicial magistrate,-9998 unclear,...,-9999,-9998,1907.0,5487.0,25,2014-01-03,2014-10-28,2014-01-16,2014-09-30,2014-10-28
2,01-01-01-201908000052013,50.0,50.0,2013,1,1,1,MHNB030000212013,chief judicial magistrate,0 male,...,-9999,0,1919.0,5148.0,25,2013-01-02,2013-01-02,2013-01-02,2013-01-02,2013-01-02
3,01-01-01-201908000052014,92.0,93.0,2014,1,1,1,MHNB030001302014,chief judicial magistrate,0 male,...,-9999,0,1907.0,4366.0,25,2014-01-02,2014-06-10,2014-02-12,2014-05-29,2014-06-10
4,01-01-01-201908000062012,92.0,92.0,2012,1,1,1,MHNB030000182012,chief judicial magistrate,-9998 unclear,...,-9999,1,1849.0,3035.0,25,2012-01-05,2012-01-12,2012-01-09,2012-01-09,2012-01-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3100892,30-02-06-201400000992011,NaN,98452.0,2011,30,2,6,GASG020016602011,criminal cases,0 male,...,-9999,-9999,2870.0,4126.0,30,2011-04-19,2013-02-18,2011-07-26,2013-02-14,2013-02-18
3100893,30-02-06-201400001002011,NaN,98462.0,2011,30,2,6,GASG020016612011,criminal cases,0 male,...,-9999,-9999,2870.0,423.0,33,2011-04-19,2016-12-13,2011-07-14,2016-12-13,2016-12-13
3100894,30-02-06-201400001042011,NaN,98364.0,2011,30,2,6,GASG020017162011,criminal cases,0 male,...,0,-9999,2870.0,1547.0,26,2011-04-26,NaN,2011-06-06,2019-01-08,2019-03-07
3100895,30-02-06-201400001092011,NaN,98452.0,2011,30,2,6,GASG020018622011,criminal cases,0 male,...,-9999,-9999,2870.0,4126.0,30,2011-05-03,2012-08-30,2011-07-14,2012-08-21,2012-08-30


### Getting Judge genders

In [8]:
judge_case=pd.merge(judge_case,judge_gender,on='ddl_decision_judge_id')

In [9]:
judge_case

,ddl_case_id,ddl_filing_judge_id,ddl_decision_judge_id,year,state_code_x,dist_code_x,court_no_x,cino,judge_position_x,female_defendant,...,date_first_list,date_last_list,date_next_list,state_code_y,dist_code_y,court_no_y,judge_position_y,female_judge,start_date,end_date
0,01-01-01-201908000012013,50.0,50.0,2013,1,1,1,MHNB030000112013,chief judicial magistrate,0 male,...,2013-01-01,2013-01-01,2013-01-01,1,1,2,chief judicial magistrate,0 nonfemale,01-10-2011,10-06-2013
1,01-01-01-201908000052013,50.0,50.0,2013,1,1,1,MHNB030000212013,chief judicial magistrate,0 male,...,2013-01-02,2013-01-02,2013-01-02,1,1,2,chief judicial magistrate,0 nonfemale,01-10-2011,10-06-2013
2,01-01-01-201908000092012,50.0,50.0,2012,1,1,1,MHNB030000552012,chief judicial magistrate,-9998 unclear,...,2012-01-21,2012-02-01,2012-02-02,1,1,2,chief judicial magistrate,0 nonfemale,01-10-2011,10-06-2013
3,01-01-01-201908000132013,50.0,50.0,2013,1,1,1,MHNB030001882013,chief judicial magistrate,-9998 unclear,...,2013-02-04,2013-02-20,2013-02-28,1,1,2,chief judicial magistrate,0 nonfemale,01-10-2011,10-06-2013
4,01-01-01-201908000192012,50.0,50.0,2012,1,1,1,MHNB030001272012,chief judicial magistrate,0 male,...,2012-02-02,2012-02-02,2012-02-02,1,1,2,chief judicial magistrate,0 nonfemale,01-10-2011,10-06-2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2979524,30-02-06-201400000682011,NaN,98452.0,2011,30,2,6,GASG020010982011,criminal cases,0 male,...,2011-04-19,2012-06-16,2012-07-07,30,2,6,criminal cases,0 nonfemale,04-06-2012,31-05-2013
2979525,30-02-06-201400000992011,NaN,98452.0,2011,30,2,6,GASG020016602011,criminal cases,0 male,...,2011-07-26,2013-02-14,2013-02-18,30,2,6,criminal cases,0 nonfemale,04-06-2012,31-05-2013
2979526,30-02-06-201400001092011,NaN,98452.0,2011,30,2,6,GASG020018622011,criminal cases,0 male,...,2011-07-14,2012-08-21,2012-08-30,30,2,6,criminal cases,0 nonfemale,04-06-2012,31-05-2013
2979527,30-02-06-201400001242011,NaN,98452.0,2011,30,2,6,GASG020020212011,criminal cases,0 male,...,2011-07-26,2012-08-07,2012-08-28,30,2,6,criminal cases,0 nonfemale,04-06-2012,31-05-2013


### Selecting relevant columns

In [10]:
judge_case=judge_case[['type_name','judge_position_x','state_code_x','dist_code_x','female_judge','female_defendant','female_adv_def','female_adv_pet','disp_name']]
judge_case=judge_case.dropna()
display(judge_case)

,type_name,judge_position_x,state_code_x,dist_code_x,female_judge,female_defendant,female_adv_def,female_adv_pet,disp_name
0,1919.0,chief judicial magistrate,1,1,0 nonfemale,0 male,-9999,0,25
1,1919.0,chief judicial magistrate,1,1,0 nonfemale,0 male,-9999,0,25
2,1849.0,chief judicial magistrate,1,1,0 nonfemale,-9998 unclear,0,0,25
3,1919.0,chief judicial magistrate,1,1,0 nonfemale,-9998 unclear,-9999,0,25
4,1849.0,chief judicial magistrate,1,1,0 nonfemale,0 male,-9999,0,25
...,...,...,...,...,...,...,...,...,...
2979524,2870.0,criminal cases,30,2,0 nonfemale,0 male,-9999,-9999,33
2979525,2870.0,criminal cases,30,2,0 nonfemale,0 male,-9999,-9999,30
2979526,2870.0,criminal cases,30,2,0 nonfemale,0 male,-9999,-9999,30
2979527,2870.0,criminal cases,30,2,0 nonfemale,0 male,-9999,-9999,33


### Grouping Data into 3 categories -- convicted, acquitted and others since for going into the details of verdicts, we'll need more information than just genders or state

In [11]:
#changing all others to 1 except for acquitted and convicted
judge_case['disp_name'] = judge_case['disp_name'].mask(judge_case['disp_name'] < 4, 1)
judge_case['disp_name'] = judge_case['disp_name'].mask(judge_case['disp_name'] >19, 1)
judge_case['disp_name'] = judge_case['disp_name'].mask((judge_case['disp_name'] < 19) & (judge_case['disp_name'] > 4), 1)
display(judge_case.head())

,type_name,judge_position_x,state_code_x,dist_code_x,female_judge,female_defendant,female_adv_def,female_adv_pet,disp_name
0,1919.0,chief judicial magistrate,1,1,0 nonfemale,0 male,-9999,0,1
1,1919.0,chief judicial magistrate,1,1,0 nonfemale,0 male,-9999,0,1
2,1849.0,chief judicial magistrate,1,1,0 nonfemale,-9998 unclear,0,0,1
3,1919.0,chief judicial magistrate,1,1,0 nonfemale,-9998 unclear,-9999,0,1
4,1849.0,chief judicial magistrate,1,1,0 nonfemale,0 male,-9999,0,1


### Adding labels to categorical variables

In [12]:
#adding labels
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

judge_case['judge_position_x'] = le.fit_transform(judge_case['judge_position_x'])
judge_case['female_defendant'] = le.fit_transform(judge_case['female_defendant'])
judge_case['female_adv_def'] = le.fit_transform(judge_case['female_adv_def'])
judge_case['female_adv_pet'] = le.fit_transform(judge_case['female_adv_pet'])
judge_case['female_judge'] = le.fit_transform(judge_case['female_judge'])


### Testing the above process

In [13]:
display(judge_case[judge_case.disp_name==19])

,type_name,judge_position_x,state_code_x,dist_code_x,female_judge,female_defendant,female_adv_def,female_adv_pet,disp_name
34,5487.0,48,1,1,1,2,0,0,19
35,5487.0,48,1,1,1,2,0,0,19
36,5487.0,48,1,1,1,2,0,0,19
37,5487.0,48,1,1,1,2,0,0,19
38,5487.0,48,1,1,1,2,0,0,19
...,...,...,...,...,...,...,...,...,...
2978683,820.0,79,29,10,2,2,1,1,19
2978753,915.0,79,29,10,2,3,0,1,19
2979101,682.0,79,29,10,2,2,0,1,19
2979283,915.0,79,29,10,1,3,2,1,19


In [15]:
from sklearn import metrics

## **TRAINING A NEURAL NETWORK**

## Importing relevant modules and defining dataframe

In [52]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd 
import seaborn as sns
# Keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import SGD, Adam, Adadelta, RMSprop
import keras.backend as K
# Train-Test
from sklearn.model_selection import train_test_split
# Scaling data
from sklearn.preprocessing import StandardScaler
# Classification Report
from sklearn.metrics import classification_report
from keras.utils.np_utils import to_categorical
#changing judge_case name for ease
df=judge_case

## Changing labels for model training

In [53]:
#changing labels from 1,4 and 9 to 0,1 and 2
others = [1]
convicted = [4]
acquitted = [19]

In [54]:
df.loc[df["disp_name"].isin(others), "disp_name"] = 0

df.loc[df["disp_name"].isin(convicted), "disp_name"] = 1

df.loc[df["disp_name"].isin(acquitted), "disp_name"] = 2

## Checking the above process

In [55]:
display(df)

,type_name,judge_position_x,state_code_x,dist_code_x,female_judge,female_defendant,female_adv_def,female_adv_pet,disp_name
0,1919.0,48,1,1,1,2,0,2,0
1,1919.0,48,1,1,1,2,0,2,0
2,1849.0,48,1,1,1,0,2,2,0
3,1919.0,48,1,1,1,0,0,2,0
4,1849.0,48,1,1,1,2,0,2,0
...,...,...,...,...,...,...,...,...,...
2979524,2870.0,56,30,2,1,2,0,0,0
2979525,2870.0,56,30,2,1,2,0,0,0
2979526,2870.0,56,30,2,1,2,0,0,0
2979527,2870.0,56,30,2,1,2,0,0,0


## Obtaining the training data by dropping the label column and scaling features

In [56]:
x = df.drop("disp_name", axis = 1)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x = pd.DataFrame(sc.fit_transform(x))
y = df["disp_name"]

## Changing to categorical prediction

In [57]:
y_cat = to_categorical(y)

## Splitting Data into training and test set. Test size was determined by the best output in various trials

In [58]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x.values, y_cat, test_size=0.2)

### **Building the neural Network**
Some points to clarify:
1. Using relu activation function for the first 2 layers and softmax instead of sigmoid for the last. Sigmoid gives values between 0 and 1, thus it can be used for binary classification and not multiple classes
2. Dropout is used for reducing overfitting. It removes some neurons from the parent network in every iteration creating a new network to train
3. Since we're doing multi class classification, instead of binary cross-entropy, we're using categorical cross-entropy

In [59]:
model = Sequential()
model.add(Dense(60, input_shape = (8,), activation = "relu"))
model.add(Dense(15, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(3, activation = "softmax"))
model.compile(Adam(lr = 0.01), "categorical_crossentropy", metrics = ["accuracy"])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 60)                540       
_________________________________________________________________
dense_5 (Dense)              (None, 15)                915       
_________________________________________________________________
dropout (Dropout)            (None, 15)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 48        
Total params: 1,503
Trainable params: 1,503
Non-trainable params: 0
_________________________________________________________________


/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


## Fitting the model

In [60]:
model.fit(x_train, y_train, verbose=1, epochs=10)


Epoch 1/10
74489/74489 [==============================] - 194s 3ms/step - loss: 0.1411 - accuracy: 0.9615
Epoch 2/10
74489/74489 [==============================] - 197s 3ms/step - loss: 0.1393 - accuracy: 0.9615
Epoch 3/10
74489/74489 [==============================] - 191s 3ms/step - loss: 0.1393 - accuracy: 0.9615
Epoch 4/10
74489/74489 [==============================] - 191s 3ms/step - loss: 0.1404 - accuracy: 0.9615
Epoch 5/10
74489/74489 [==============================] - 200s 3ms/step - loss: 0.1407 - accuracy: 0.9616
Epoch 6/10
74489/74489 [==============================] - 199s 3ms/step - loss: 0.1409 - accuracy: 0.9616
Epoch 7/10
74489/74489 [==============================] - 193s 3ms/step - loss: 0.1409 - accuracy: 0.9616
Epoch 8/10
74489/74489 [==============================] - 193s 3ms/step - loss: 0.1407 - accuracy: 0.9616
Epoch 9/10
74489/74489 [==============================] - 194s 3ms/step - loss: 0.1415 - accuracy: 0.9616
Epoch 10/10
74489/74489 [=====================

### This accuracy was obtained once while training the neural net, However, running multiple I obtained a range of accuracies but none close to this. Therefore, I'm reporting the highest accuracy obtained from this code.

source: https://www.analyticsvidhya.com/blog/2021/07/performing-multi-class-classification-on-fifa-dataset-using-keras/